In [23]:
## Tropospheric delay correction using GACOS from New Castle U
## for geocoded dataset
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize
from pysar.objects import timeseries
from pysar.utils import ptime, readfile, utils as ut

ztd_dir = os.path.expanduser('/media/lei/disk2/ErtaAle/20190605/SenAT14/WEATHER/GACOS')
proj_dir = os.path.expanduser('/media/lei/disk2/ErtaAle/20190605/SenAT14/PYSAR')

out_file = os.path.join(proj_dir, 'INPUTS/GACOS.h5')
ts_file = os.path.join(proj_dir, 'timeseries.h5')
geom_file = os.path.join(proj_dir, 'INPUTS/geometryRadar.h5')

In [24]:
def get_geo_delay(fztd, geo_box, atr, inc_angle):
    meta = readfile.read_attribute(fztd)
    coord = ut.coordinate(meta)
    box = coord.box_geo2pixel(geo_box)
    phs = readfile.read(fztd, box=box)[0]
    phs[phs == 0] = np.nan

    # interpolation
    out_shape = (int(atr['LENGTH']), int(atr['WIDTH']))
    phs = resize(phs, out_shape, order=1, mode='edge', anti_aliasing=True)

    phs /= np.cos(inc_angle * np.pi / 180)
    phs -= phs[int(atr['REF_Y']), int(atr['REF_X'])]
    phs *= -1
    return phs

In [25]:
# 1. get time-series file info
obj = timeseries(ts_file)
obj.open()
atr = readfile.read_attribute(ts_file)

# get box_box
pixel_box = (0, 0, int(atr['WIDTH']), int(atr['LENGTH']))
geo_box = ut.coordinate(atr).box_pixel2geo(pixel_box)

# get inc_angle
inc_angle = readfile.read(geom_file, datasetName='incidenceAngle')[0]

# get grib file list
fztd_list = [os.path.join(ztd_dir, '{}.ztd'.format(i)) for i in obj.dateList]

open timeseries file: timeseries.h5


In [7]:
# calculate deley time-series
num_date = obj.numDate
tropo = np.zeros((num_date, obj.length, obj.width), dtype=np.float32)
prog_bar = ptime.progressBar(maxValue=num_date)
for i in range(num_date):
    fztd = fztd_list[i]
    tropo[i, :, :] = get_geo_delay(fztd, geo_box, atr, inc_angle)
    prog_bar.update(i+1, suffix=os.path.basename(fztd))
prog_bar.close()
tropo -= np.tile(tropo[0, :, :], (num_date, 1, 1))

# write deley to h5 file
obj_out = timeseries(out_file)
obj_out.write2hdf5(data=tropo,
                   dates=obj.dateList,
                   metadata=atr,
                   refFile=ts_file)

[==================================================] 20190425.ztd    0s /     0s 
create timeseries HDF5 file: /media/lei/disk2/ErtaAle/20190605/SenAT14/PYSAR/INPUTS/GACOS.h5 with w mode
create dataset /timeseries of float32    in size of (35, 900, 500) with compression=None
create dataset /dates      of |S8        in size of (35,)
create dataset /bperp      of float32    in size of (35,)
finished writing to /media/lei/disk2/ErtaAle/20190605/SenAT14/PYSAR/INPUTS/GACOS.h5


'/media/lei/disk2/ErtaAle/20190605/SenAT14/PYSAR/INPUTS/GACOS.h5'

In [26]:
# correct time-series
work_dir = '/media/lei/disk2/ErtaAle/20190605/SenAT14/PYSAR'
os.chdir(work_dir)
print('Go to directory', work_dir)

!diff.py timeseries.h5 INPUTS/GACOS.h5 -o timeseries_GACOS.h5
#!remove_ramp.py timeseries_GACOS.h5 -m maskTempCoh.h5 -s quadratic
!dem_error.py timeseries_GACOS.h5 -t pysarApp_template.txt

Go to directory /media/lei/disk2/ErtaAle/20190605/SenAT14/PYSAR
timeseries.h5 - ['INPUTS/GACOS.h5'] --> timeseries_GACOS.h5
input files are: timeseries and timeseries
open timeseries file: timeseries.h5
open timeseries file: GACOS.h5
consider different reference pixel
reading timeseries data from file: timeseries.h5 ...
create timeseries HDF5 file: timeseries_GACOS.h5 with w mode
create dataset /timeseries of float32    in size of (29, 900, 500) with compression=None
create dataset /dates      of |S8        in size of (29,)
create dataset /bperp      of float32    in size of (29,)
finished writing to timeseries_GACOS.h5
read options from template file: pysarApp_template.txt
read mean incidenceAngle,slantRangeDistance,bperp value from timeseries file
near   incidence angle : 30.8616 degree
far    incidence angle : 33.8532 degree
center incidence angle : 32.3574 degree
center range : 810604.65 m
near   range : 798980.14 m
far    range : 822229.17 m
open timeseries file: timeseries_GACOS.

In [28]:
!view.py timeseries_GACOS_demErr.h5 -v -25 15 --nodisplay
!view.py timeseries_ECMWF_demErr.h5 -v -25 15 --nodisplay

run view.py in PySAR release version v1.1.0-14, release date 2019-05-01
input file is timeseries file: /media/lei/disk2/ErtaAle/20190605/SenAT14/PYSAR/timeseries_GACOS_demErr.h5 in float32 format
file size in y/x: (900, 500)
num of datasets in file timeseries_GACOS_demErr.h5: 29
datasets to exclude (0):
[]
datasets to display (29):
['timeseries-20170204', 'timeseries-20170914', 'timeseries-20180112', 'timeseries-20180124', 'timeseries-20180205', 'timeseries-20180217', 'timeseries-20180301', 'timeseries-20180313', 'timeseries-20180325', 'timeseries-20180430', 'timeseries-20180524', 'timeseries-20180605', 'timeseries-20180617', 'timeseries-20180629', 'timeseries-20180816', 'timeseries-20181003', 'timeseries-20181015', 'timeseries-20181027', 'timeseries-20181120', 'timeseries-20181202', 'timeseries-20181214', 'timeseries-20181226', 'timeseries-20190107', 'timeseries-20190119', 'timeseries-20190224', 'timeseries-20190320', 'timeseries-20190401', 'timeseries-20190413', 'timeseries-20190425'